In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense,Activation,Dropout
from keras.utils import to_categorical
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import seaborn as sbn
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.wrappers.scikit_learn import KerasClassifier


In [ ]:
data_directory = "My Drive/Veri/"

In [ ]:
no_tumor = os.listdir(data_directory + "no/")
yes_tumor = os.listdir(data_directory + "yes/")

In [ ]:
no_tumor

In [ ]:
path = "no0.jpg"
dataset=[]
label=[]
Input_Size = 64

In [ ]:
path.split(".")[1]

'jpg'

In [ ]:
for i, image_name in enumerate(no_tumor):
  if(image_name.split(".")[1] == "jpg"):
    img = cv2.imread(data_directory+ "no/" +image_name)
    img = Image.fromarray(img,"RGB")
    img = img.resize((Input_Size,Input_Size))
    img = np.array(img)
    dataset.append(img)
    label.append(0)

In [ ]:
for i, image_name in enumerate(yes_tumor):
  if(image_name.split(".")[1] == "jpg"):
    img = cv2.imread(data_directory + "yes/" + image_name)
    img = Image.fromarray(img,"RGB")
    img = img.resize((Input_Size,Input_Size))
    dataset.append(np.array(img))
    label.append(1)

In [ ]:
dataset = np.array(dataset)
label = np.array(label)

In [ ]:
label.shape

(3000,)

In [ ]:
dataset.shape

(3000, 64, 64, 3)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(dataset,label,test_size=0.3,train_size=0.7,random_state=0)

In [ ]:
x_train.shape

(2100, 64, 64, 3)

In [ ]:
x_train = normalize(x_train,axis=1)
x_test= normalize(x_test,axis=1)

Model Building

In [ ]:
model = Sequential()

model.add(Conv2D( 32 , (3,3) , input_shape=(Input_Size,Input_Size,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D( 64 , (3,3) , kernel_initializer= "he_uniform"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D( 128 , (3,3) , kernel_initializer= "he_uniform"))
model.add(Activation("relu"))
model.add(Dropout(0.1))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer= "adam" , metrics = ["accuracy"])

In [ ]:
model.fit(x_train,y_train,
          batch_size=32,
          verbose=1,
          epochs = 10,
          validation_data=(x_test,y_test),
          shuffle=False)

Epoch 1/10
66/66 [==============================] - 10s 17ms/step - loss: 0.5435 - accuracy: 0.7324 - val_loss: 0.4696 - val_accuracy: 0.8133
Epoch 2/10
66/66 [==============================] - 1s 10ms/step - loss: 0.3655 - accuracy: 0.8424 - val_loss: 0.2893 - val_accuracy: 0.9056
Epoch 3/10
66/66 [==============================] - 1s 10ms/step - loss: 0.2718 - accuracy: 0.8814 - val_loss: 0.2152 - val_accuracy: 0.9178
Epoch 4/10
66/66 [==============================] - 1s 10ms/step - loss: 0.1908 - accuracy: 0.9343 - val_loss: 0.1670 - val_accuracy: 0.9389
Epoch 5/10
66/66 [==============================] - 1s 10ms/step - loss: 0.1256 - accuracy: 0.9562 - val_loss: 0.1286 - val_accuracy: 0.9433
Epoch 6/10
66/66 [==============================] - 1s 10ms/step - loss: 0.0920 - accuracy: 0.9719 - val_loss: 0.1277 - val_accuracy: 0.9522
Epoch 7/10
66/66 [==============================] - 1s 10ms/step - loss: 0.0606 - accuracy: 0.9819 - val_loss: 0.1395 - val_accuracy: 0.9544
Epoch 8/10
6

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 activation (Activation)     (None, 62, 62, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 29, 29, 64)        0         
                                                        

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('/content/drive/MyDrive/Tumor_CNN')

mkdir: cannot create directory ‘saved_model’: Operation not supported


In [ ]:
pred = model.predict_generator(x_test,verbose=0)

pred[ pred <= .5] = 0
pred[ pred > .5] = 1

<ipython-input-21-ddd79674e6fa>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(x_test,verbose=0)


In [ ]:
confusion_matrix(y_test , pred)

array([[479,  14],
       [ 11, 396]])

In [ ]:
model.evaluate(x_train,y_train,verbose=0)

[0.014420244842767715, 0.9961904883384705]

In [ ]:
# Evaluate the model
loss, acc = model.evaluate(x_test,y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

29/29 - 0s - loss: 0.1346 - accuracy: 0.9722 - 195ms/epoch - 7ms/step
Restored model, accuracy: 97.22%
